In [0]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt


from tensorflow.examples.tutorials.mnist.input_data import read_data_sets as load_mnist
mnist = load_mnist("MNIST_data/", one_hot=True)

train_image = mnist.train.images.reshape(-1, 28, 28, 1)
test_image = mnist.test.images.reshape(-1, 28, 28, 1)

train_label = np.array(mnist.train.labels, dtype=np.bool)
test_label = np.array(mnist.test.labels, dtype=np.bool)

In [0]:
epoch_cnt = 1
batch_size = 16
learning_rate = 0.001
hidden_num = 64
kernel_size = 3

In [0]:
X = tf.placeholder(shape=(None, 28, 28, 1), dtype=tf.float32)
y = tf.placeholder(shape=(None, 10), dtype=tf.int32)

# Conv -> Pool -> Conv -> Pool -> Output
out = tf.layers.conv2d(inputs= X,
                       filters= hidden_num,
                       kernel_size= kernel_size,
                       activation=tf.nn.elu,
                       kernel_initializer=tf.random_normal_initializer())

out = tf.layers.max_pooling2d(inputs = out,
                              pool_size= 2,
                              strides = 2)

out = tf.layers.conv2d(inputs= out,
                       filters= hidden_num,
                       kernel_size= kernel_size,
                       activation=tf.nn.elu,
                       kernel_initializer=tf.random_normal_initializer())

out = tf.layers.max_pooling2d(inputs = out,
                              pool_size= 2,
                              strides = 2)

out = tf.layers.flatten(out)
out = tf.layers.dense(inputs= out, units = 10)

In [0]:
# 에러 함수
loss = tf.losses.softmax_cross_entropy(onehot_labels= y, logits= out)

# 미분 및 Backpropagation
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

error_list = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    fetch_dict = {"optim": optimizer, "loss": loss}
    
    for epoch in range(epoch_cnt):
        rand_idx = np.random.permutation(len(train_image))
        for step in range(0, len(rand_idx), batch_size):
            cur_idx = rand_idx[step:step+batch_size]
            result = sess.run(fetch_dict, feed_dict={X: train_image[cur_idx], y: train_label[cur_idx]})
            error_list.append(result["loss"])
            if step%1000 ==0:
                print("Epoch %d"%epoch, "Step %d"%step, "Loss %f"%result["loss"])

In [0]:
import matplotlib.pyplot as plt
plt.plot(error_list)